In [1]:
!wget https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json

--2024-01-31 22:18:05--  https://storage.googleapis.com/tensorflow-1-public/course3/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.207, 172.253.123.207, 142.250.98.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json’

sarcasm.json        100%[===================>]   5.38M  --.-KB/s    in 0.04s   

2024-01-31 22:18:05 (126 MB/s) - ‘sarcasm.json’ saved [5643545/5643545]



In [7]:
import json

with open('sarcasm.json') as file:
  data = json.load(file)

sentences = []
labels = []

for row in data:
  sentences.append(row['headline'])
  labels.append(row['is_sarcastic'])

print(f'sentences  {len(sentences)} labels {len(labels)}')

sentences  26709 labels 26709


In [10]:
split = int(len(labels) * 0.8)
train_sentences = sentences[:split]
test_sentences = sentences[split:]
train_labels = labels[:split]
test_labels = labels[split:]



In [1]:

import matplotlib.pyplot as plt

def plot_graph(history, string):

    plt, axs = plt.subplots(2)
    axs[0].plot(history.history['loss'], label=string +'loss')
    axs[0].plot(history.history['val_loss'], label= string+'validation loss')
    axs[0].legend()
    axs[1].plot(history.history['accuracy'], label=string+'accuracy')
    axs[1].plot(history.history['val_accuracy'], label=string+'validation accuracy')
    axs[1].legend()

In [14]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size= 10000
max_length= 120
trunc_type='post'
padding_type='post'
oov_tok='<OOV>'

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded_sequences =pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded_sequences =pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
import tensorflow as tf

In [ ]:
#lstm

embedding_dim_lstm = 16
lstm_dim = 16
dense_dim = 32

model_lstm = tf.keras.models.Sequential([
    tf.keras.Emabedding(vocab_size, embedding_dim_lstm, input_length=max_length),
    tf.keras.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

epochs_lstm = 10
history_lstm = model_lstm.fit(train_padded_sequences, train_labels, epochs=epochs_lstm,
                              validation_data=(test_padded_sequences, test_labels))

plot_graph(history_lstm, "lstm")

In [ ]:
#Conv

embedding_dim_conv = 16
filters=64
kernel_size=3
dense_dim_conv = 32

model_conv = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim_conv),
    tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'),
    tf.keras.layers.GlobalMaxPool1D(),
    tf.keras.layers.Dense(dense_dim_conv, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_conv.summary()

epochs_conv = 10
history_conv = model_conv.fit(train_padded_sequences, train_labels, epochs=epochs_conv,
                              validation_data=(test_padded_sequences, test_labels))

plot_graph(history_conv, "conv")